# Empire State Child Credit

The Empire State Child Credit, New York's Child Tax Credit, provides a 33 percent match to the federal Child Tax Credit, or a minimum of \$100 per qualifying child if the filer has income below the CTC phase-out threshold.
Qualifying children are children eligible for the federal CTC and who are at least four years old.

## Examples


In [2]:
from openfisca_us import IndividualSim, Microsimulation
import pandas as pd
import plotly.express as px


def make_eitc(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="NY")
    sim.vary("employment_income")
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            ny_ctc=sim.calc("ny_ctc")[0].round(),
            mtr=-sim.deriv("ny_ctc", "employment_income", wrt_target="head"),
            adults=adults,
            children=children,
        )
    )


# Make a table of EITCs for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_eitc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    mtr="Marginal tax rate of Empire State Child Credit",
    adults="Adults",
    children="Children",
    ny_ctc="Empire State Child Credit",
)

fig = px.line(
    df,
    "employment_income",
    "ny_ctc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Empire State Child Credit",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat="$,", height=600, width=800
)
fig.show()

The Empire State Child Credit creates marginal tax rates ranging from -13.5 percent to +6.3 percent, plus downward spikes corresponding to the NY household credit cliffs.


In [3]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Empire State Child Credit marginal tax rate",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat=".1%")
fig.show()

## Budgetary impact

Applying the New York EITC logic to the 2020 Current Population Survey March Supplement shows that the program provided an estimated \$490 million in 2020; however, because it interacts with other programs, the cost of its introduction is \$483 million.


In [4]:
from openfisca_us.model_api import *


def budgetary_impacts(
    variable: str, data_year: int = 2020, policy_year: int = 2022
) -> Tuple:
    """Calculates the budgetary impact of a variable for a given policy year.

    Args:
        variable (str): The variable to calculate the budgetary impact of.
        data_year (int): The year of the data to use.
        policy_year (int): The year of the policy to use.

    Returns:
        Tuple: A tuple of the total cost and budgetary impact of repealing the variable,
        respectively, in millions.
    """

    class ignore_reported(Reform):
        def apply(self):
            self.neutralize_variable("spm_unit_net_income_reported")

    class neutralize(Reform):
        def apply(self):
            self.neutralize_variable(variable)

    sim = Microsimulation(ignore_reported, year=data_year)
    sim_neutralized = Microsimulation(
        (ignore_reported, neutralize), year=data_year
    )
    program_value = sim.calc(variable, period=policy_year).sum()
    baseline_net_income = sim.calc(
        "spm_unit_net_income", period=policy_year
    ).sum()
    neutralized_net_income = sim_neutralized.calc(
        "spm_unit_net_income", period=policy_year
    ).sum()
    budgetary_impact = neutralized_net_income - baseline_net_income
    return round(program_value / 1e6), round(budgetary_impact / 1e6)


budgetary_impacts("ny_ctc")

(490, -483)